In [257]:
## Actividad Integradora 2
 
#Estefanía Pérez Yeo - A01639270
#Vanessa Méndez Palacios - A01639925
#Salvador Fernando Camacho Hernández - A01634777
#Paola Elizabeth Gómez Martínez - A01633772
#Héctor Robles Villarreal - A01634105


import agentpy as ap
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random
from time import time
import json

In [258]:
parameters = {
    'm': 20, # Altura del grid
    'n': 20, # Longitud del grid
    'k': 5, # Número de carros
    'tMax': 100, # Tiempo máximo de ejecucción
    'getJson' : True
}
class IntersectionModel(ap.Model):
    
    def setup(self):
        
        global calle_v1, calle_v2, calle_v3, calle_v4, calle_v5, calle_v6, calleV
        global calle_h1, calle_h2, calle_h3, calle_h4, calle_h5, calle_h6, calleH
        
        #Calle Superior
        calle_v1 = [(n, int(self.p.n/2) - 1) for n in range(0, int(self.p.m/2), 1)]
        calle_v2 = [(n, int(self.p.n/2)) for n in range(0, int(self.p.m/2), 1)]
        calle_v3 = [(n, int(self.p.n/2) + 1) for n in range(0, int(self.p.m/2), 1)]
        
        #Calle Inferior
        calle_v4 = [(n, int(self.p.n/2) - 1) for n in range(int(self.p.m/2), self.p.m , 1)]
        calle_v5 = [(n, int(self.p.n/2)) for n in range(int(self.p.m/2), self.p.m , 1)]
        calle_v6 = [(n, int(self.p.n/2) + 1) for n in range(int(self.p.m/2), self.p.m , 1)]
        
        #Calle Izquierda
        calle_h1 = [(int(self.p.m/2) - 1, n) for n in range(0, int(self.p.n/2), 1)]
        calle_h2 = [(int(self.p.m/2), n) for n in range(0, int(self.p.n/2), 1)]
        calle_h3 = [(int(self.p.m/2) + 1, n) for n in range(0, int(self.p.n/2), 1)]
        
        #Calle Derecha
        calle_h4 = [(int(self.p.m/2) - 1, n) for n in range(int(self.p.n/2), self.p.n, 1)]
        calle_h5 = [(int(self.p.m/2), n) for n in range(int(self.p.n/2 ) + 2, self.p.n, 1)]
        calle_h6 = [(int(self.p.m/2) + 1, n) for n in range(int(self.p.n/2), self.p.n, 1)]
        
        callesH = calle_h1+ calle_h2+ calle_h3+ calle_h4+ calle_h5+ calle_h6
        callesV = calle_v1+ calle_v2+ calle_v3+ calle_v4+ calle_v5 + calle_v6
        
        # Generar posiciones de inicio aleatorias
        random.seed(time())
        self.pos=[(1, round(self.p.n / 2) - 1), (self.p.m - 2, round(self.p.n / 2)), (round(self.p.m / 2), 1), (round(self.p.m / 2)-1, self.p.n - 2)]
        for i in range(0,self.p.k):
            self.pos.append(self.pos[random.randint(0,3)])
       
        # Crear agentes
        cars = self.agents = ap.AgentList(self, self.p.k)
        
        # Crear grid
        self.street = ap.Grid(self, (self.p.m, self.p.n), track_empty=True)
        self.street.add_agents(cars, positions=self.pos[4:14], empty=True)
        
        
        self.camino = ap.AgentList(self,len(cars))
        
        self.camino.condition = 1
        
        self.agents.condition = 0
        self.agents.orientation = 'x'
        
        
        #Creación del JSON
        if self.p.getJson == True:
            self.steps = 0
            self.data = {}
            self.data['steps'] = []
        
    
    def step(self):
        for i in self.agents:
            
            if(self.street.positions[i] == self.pos[0]):
                i.orientation = 'n' 
            if(self.street.positions[i] == self.pos[1]):
                i.orientation = 's'
            if(self.street.positions[i] == self.pos[2]):
                i.orientation = 'w'
            if(self.street.positions[i] == self.pos[3]):
                i.orientation = 'e'
            
            
            if(self.street.positions[i] == (self.p.m-2, round(self.p.n/2)-1) # Final 'n' posición
              or self.street.positions[i] == (1, round(self.p.n/2)) # Final 's' posición
              or self.street.positions[i] == (round(self.p.m/2), self.p.n-2) # Final 'w' posición
              or self.street.positions[i] == (round(self.p.m/2)-1, 1)): # Final 'e' posición
                i.orientation = 'x'
                self.street.move_to(i, self.pos[random.randint(0,3)])
            
            #Moverse
            if(i.orientation == 'n' 
                and not self.street.agents[self.street.positions[i][0]+1, self.street.positions[i][1]].condition == True
                and not self.street.agents[self.street.positions[i][0]+2, self.street.positions[i][1]].condition == True):
                    self.street.move_by(i, (1, 0))
                        
            if(i.orientation == 's'
               and not self.street.agents[self.street.positions[i][0]-1, self.street.positions[i][1]].condition == True
               and not self.street.agents[self.street.positions[i][0]-2, self.street.positions[i][1]].condition == True):
                    self.street.move_by(i, (-1, 0))
                    
            if(i.orientation == 'w'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+2].condition == True):
                self.street.move_by(i, (0, 1))
                
            if(i.orientation == 'e'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-2].condition == True):
                self.street.move_by(i, (0, -1))  
                
            
            #Girar
            if(i.orientation == 'n'
                and not self.street.agents[self.street.positions[i][0]+1, self.street.positions[i][1]].condition == True
                #and not self.street.agents[self.street.positions[i][0]+2, self.street.positions[i][1]].condition == True
                and ((self.street.positions[i] in calle_h2) or (self.street.positions[i] in calle_h5))):
                self.street.move_by(i,[0,1])
                i.orientation = 'w'
                
            elif(i.orientation == 's' 
               and not self.street.agents[self.street.positions[i][0]-1, self.street.positions[i][1]].condition == True
               and not self.street.agents[self.street.positions[i][0]-2, self.street.positions[i][1]].condition == True
                and ((self.street.positions[i] in calle_h2) or (self.street.positions[i] in calle_h5))):
                self.street.move_by(i,[0,-1])
                i.orientation = 'e'
                
            elif(i.orientation == 'w'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+1].condition == True
               #and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+2].condition == True
                 and ((self.street.positions[i] in calle_v2) or (self.street.positions[i] in calle_v5))):
                self.street.move_by(i, (1, 0))
                i.orientation = 'n'
                
            elif(i.orientation == 'e'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-2].condition == True
               and ((self.street.positions[i] in calle_v2) or (self.street.positions[i] in calle_v5))):
                self.street.move_by(i,[-1,0])
                i.orientation = 's'
                
        
        camino_pendiente = self.camino.select(self.camino.condition == 1)
        if self.t == self.p.tMax/2 or len(camino_pendiente) == 0:
            self.stop()
        
        
        #Añadir a JSON
        if self.p.getJson == True:
            cars_list = []
            for i in range(self.p.k):
                i
                cars_list.append({
                    'id': i,
                    'x': self.street.positions[self.agents[i]][0],
                    'y': 0,
                    'z': self.street.positions[self.agents[i]][1],
                })
            

            self.data['steps'].append({
                
                'cars': cars_list
            })

            self.steps += 1
            
    
    def end(self):
        #Escribir JSON
        if self.p.getJson == True:
            file_json = json.dumps(self.data)
            with open('datos.json', 'w') as outfile:
                outfile.write(file_json)
        return


In [259]:
def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#FF0000', 1:'#808080', 2:'#808080', None:'#00A300'}
    #0:'#744700', 1:'#3d9de6', 2:'#bcbcbc', None:'#ffffff'
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulación Intersección\n"
                             f"Tiempo: {model.t}")
fig, ax = plt.subplots()
model = IntersectionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=5))

In [260]:
model = IntersectionModel(parameters)
results = model.run()

Completed: 50 steps
Run time: 0:00:00.078125
Simulation finished
